In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import cachetools
import random 
import numpy as np
import tensorflow as tf


2024-07-07 05:57:50.278282: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 05:57:51.068898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

@cachetools.cached(cache={})
def read_video(path: str, frame_index: int = None, min_brightness: int = 10):
    video = cv2.VideoCapture(path)
    if not video.isOpened():
        raise IOError(f"Error opening video file: {path}")
    

    
    frames = []
    def normalize(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)
        img = cv2.resize(img, (480, 640), None)
        return img 
    
    while video.isOpened():
        ret, frame = video.read()
        frames.append(frame)
        if not ret:
            break
        if len(frames) >= frame_index and np.average(frame) > min_brightness:
            return normalize(frame)
    if frame_index is None:
        frame_index = len(frames) // 2
        while np.average(frames[frame_index]) < min_brightness:
            frame_index += 1
        return normalize(frames[frame_index]) 
    raise ValueError("frame not found")
REAL_FILES = (
    "1.avi",
    "2.avi",
    "HR_1.avi",
    "HR_4.avi",
)
def read_dir(path: str):
    real_images = []
    spoof_images = []
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
            r, s = read_dir(os.path.join(path, file))
            real_images.extend(r)
            spoof_images.extend(s)
        else:
            if file.endswith(".avi"):
                obj = (os.path.join(path, file))
                if file in REAL_FILES:
                    real_images.append(obj)
                    
                else:
                    spoof_images.append(obj)
    return real_images, spoof_images


def show_data(ds, rows, cols):
    random.shuffle(ds)
    for i in range(rows):
        for j in range(cols):
            ind = i * cols + j 
            plt.subplot(rows, cols, ind + 1)
            plt.imshow(ds[ind])
            plt.axis("off")


def load_data(path: str, show = False):
    @cachetools.cached(cache={})
    def inner(path):
        r, s = read_dir(path)
        real = [read_video(i, frame_index=1) for i in r]
        spoof = [read_video(i, frame_index=1) for i in s]
        return real, spoof
    real, spoof = inner(path)
    if show:
        show_data(spoof, 4, 3)
        plt.figure()
        show_data(real, 4, 3)
    ds = [(i, 0) for i in real] + [(i, 1) for i in spoof]
    random.shuffle(ds)
    x = np.array([i[0] for i in ds])
    y = np.array([i[1] for i in ds])
    return x, y

In [14]:
x_train, y_train = load_data("./datasets/train_release")
x_test, y_test = load_data("./datasets/test_release")

In [15]:
# https://openaccess.thecvf.com/content_cvpr_2018/papers/Liu_Learning_Deep_Models_CVPR_2018_paper.pdf
mobilenet = tf.keras.applications.MobileNet(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(640, 480, 3),
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)
for l in mobilenet.layers[:len(mobilenet.layers)]:
    l.trainable = False

model = tf.keras.Sequential()
model.add(mobilenet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(rate = 0.8))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



/tmp/ipykernel_223117/635910364.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = tf.keras.applications.MobileNet(


In [25]:
def add_salt_pepper(img, rate = 0.2): 
    
    w, h, _ = img.shape 
    noised_image = img.copy()
    white = (255, 255, 255)
    black = (0, 0, 0)
    for i in range(w):
        for j in range(h):
            if random.random() > rate:
                continue
            noised_image[i, j] = white if random.random() > 0.5 else black 
    return noised_image

noised_image = [add_salt_pepper(i) for i in x_train]
blurred_image = [cv2.GaussianBlur(i, (13, 13), 10) for i in x_train]

augmented_x = np.concatenate([x_train, noised_image, blurred_image])
augmented_y = np.concatenate([y_train, y_train, y_train])
ind = list(range(len(augmented_x)))
random.shuffle(ind)
x_train_final = [None for _ in range(len(augmented_x))]
y_train_final = [None for _ in range(len(augmented_x))]
for i in range(len(ind)):
    x_train_final[i] = augmented_x[ind[i]]
    y_train_final[i] = augmented_y[ind[i]]


In [28]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))
# print(len(x_train_final), len(y_train_final), len(x_train))

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 815ms/step - accuracy: 0.5959 - loss: 20.0751 - val_accuracy: 0.8389 - val_loss: 3.1569
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 331ms/step - accuracy: 0.8928 - loss: 3.5662 - val_accuracy: 0.8556 - val_loss: 3.5480
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 332ms/step - accuracy: 0.9693 - loss: 1.0266 - val_accuracy: 0.9306 - val_loss: 2.3579
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 333ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8861 - val_loss: 4.0287
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 331ms/step - accuracy: 0.9986 - loss: 0.0096 - val_accuracy: 0.8444 - val_loss: 5.5298
